## Configuração do ambiente para utilização do Spark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Fazendo download
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# Descompactando os arquivos
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

# Importando a biblioteca os
import os

# Definindo a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Definindo a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"


# instalando a findspark
!pip install -q findspark

# Importando a findspark
import findspark

# Iniciando o findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Minha Primeira Aplicação no Pyspark") \
        .getOrCreate()

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Leitura dos dados

In [ ]:
import pandas as pd
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projetos/Modelo_Credito_Application /Conjunto_de_Dados/credit_card_balance.csv')
df1.head()

SK_ID_PREV  SK_ID_CURR  MONTHS_BALANCE  AMT_BALANCE  \
0     2562384      378907              -6       56.970   
1     2582071      363914              -1    63975.555   
2     1740877      371185              -7    31815.225   
3     1389973      337855              -4   236572.110   
4     1891521      126868              -1   453919.455   

   AMT_CREDIT_LIMIT_ACTUAL  AMT_DRAWINGS_ATM_CURRENT  AMT_DRAWINGS_CURRENT  \
0                   135000                       0.0                 877.5   
1                    45000                    2250.0                2250.0   
2                   450000                       0.0                   0.0   
3                   225000                    2250.0                2250.0   
4                   450000                       0.0               11547.0   

   AMT_DRAWINGS_OTHER_CURRENT  AMT_DRAWINGS_POS_CURRENT  \
0                         0.0                     877.5   
1                         0.0                       0.0   
2                         0.0                       0.0   
3                         0.0                       0.0   
4                         0.0                   11547.0   

   AMT_INST_MIN_REGULARITY  ...  AMT_RECIVABLE  AMT_TOTAL_RECEIVABLE  \
0                 1700.325  ...          0.000                 0.000   
1                 2250.000  ...      64875.555             64875.555   
2                 2250.000  ...      31460.085             31460.085   
3                11795.760  ...     233048.970            233048.970   
4                22924.890  ...     453919.455            453919.455   

   CNT_DRAWINGS_ATM_CURRENT  CNT_DRAWINGS_CURRENT  CNT_DRAWINGS_OTHER_CURRENT  \
0                       0.0                     1                         0.0   
1                       1.0                     1                         0.0   
2                       0.0                     0                         0.0   
3                       1.0                     1                         0.0   
4                       0.0                     1                         0.0   

   CNT_DRAWINGS_POS_CURRENT  CNT_INSTALMENT_MATURE_CUM  NAME_CONTRACT_STATUS  \
0                       1.0                       35.0                Active   
1                       0.0                       69.0                Active   
2                       0.0                       30.0                Active   
3                       0.0                       10.0                Active   
4                       1.0                      101.0                Active   

   SK_DPD  SK_DPD_DEF  
0       0           0  
1       0           0  
2       0           0  
3       0           0  
4       0           0  

[5 rows x 23 columns]

In [ ]:
# Lê o arquivo Parquet
dados = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/Projetos/Modelo_Credito_Application /Conjunto_de_Dados/credit_card_balance.csv",header=True)

# Mostra os dados
dados.show()

+----------+----------+--------------+-----------+-----------------------+------------------------+--------------------+--------------------------+------------------------+-----------------------+-------------------+-------------------------+------------------------+-------------+--------------------+------------------------+--------------------+--------------------------+------------------------+-------------------------+--------------------+------+----------+
|SK_ID_PREV|SK_ID_CURR|MONTHS_BALANCE|AMT_BALANCE|AMT_CREDIT_LIMIT_ACTUAL|AMT_DRAWINGS_ATM_CURRENT|AMT_DRAWINGS_CURRENT|AMT_DRAWINGS_OTHER_CURRENT|AMT_DRAWINGS_POS_CURRENT|AMT_INST_MIN_REGULARITY|AMT_PAYMENT_CURRENT|AMT_PAYMENT_TOTAL_CURRENT|AMT_RECEIVABLE_PRINCIPAL|AMT_RECIVABLE|AMT_TOTAL_RECEIVABLE|CNT_DRAWINGS_ATM_CURRENT|CNT_DRAWINGS_CURRENT|CNT_DRAWINGS_OTHER_CURRENT|CNT_DRAWINGS_POS_CURRENT|CNT_INSTALMENT_MATURE_CUM|NAME_CONTRACT_STATUS|SK_DPD|SK_DPD_DEF|
+----------+----------+--------------+-----------+------------------

## Criação de flags para nos auxiliar na visão temporal dos dados

Nesta etapa, foi habilitado o uso do SparkSQL para permitir consultas SQL diretamente sobre o DataFrame dados. Primeiro, o DataFrame foi registrado como uma visão temporária (dados.createOrReplaceTempView("dados")), o que possibilita tratá-lo como uma tabela SQL. Em seguida, foi executada uma consulta SQL que cria novas colunas indicando se cada registro pertence aos últimos 3, 6, 12, 24 e 36 meses, com base no campo MONTHS_BALANCE. Essas colunas funcionam como marcadores temporais (flags) e serão úteis para análises que dependem de períodos específicos, como a identificação de comportamentos recentes. Por fim, o resultado foi ordenado pela coluna SK_ID_PREV e salvo em uma nova visão temporária (df_temp_01), permitindo continuar as análises com o novo conjunto de dados enriquecido.

In [ ]:
# Habilitando uso do SparkSQL
dados.createOrReplaceTempView("dados")

df_temp_01 = spark.sql("""
SELECT
    *,
      CASE
        WHEN MONTHS_BALANCE >= -3 THEN 1
        ELSE 0
    END AS ultimos_3_meses,
    CASE
        WHEN MONTHS_BALANCE >= -6 THEN 1
        ELSE 0
    END AS ultimos_6_meses,
    CASE
        WHEN MONTHS_BALANCE >= -12 THEN 1
        ELSE 0
    END AS ultimos_12_meses,
    CASE
        WHEN MONTHS_BALANCE >= -24 THEN 1
        ELSE 0
    END AS ultimos_24_meses,
    CASE
        WHEN MONTHS_BALANCE >= -36 THEN 1
        ELSE 0
    END AS ultimos_36_meses
FROM dados
ORDER BY `SK_ID_PREV`;
""")
df_temp_01.createOrReplaceTempView("df_temp_01")
df_temp_01.show()

+----------+----------+--------------+-----------+-----------------------+------------------------+--------------------+--------------------------+------------------------+-----------------------+-------------------+-------------------------+------------------------+-------------+--------------------+------------------------+--------------------+--------------------------+------------------------+-------------------------+--------------------+------+----------+---------------+---------------+----------------+----------------+----------------+
|SK_ID_PREV|SK_ID_CURR|MONTHS_BALANCE|AMT_BALANCE|AMT_CREDIT_LIMIT_ACTUAL|AMT_DRAWINGS_ATM_CURRENT|AMT_DRAWINGS_CURRENT|AMT_DRAWINGS_OTHER_CURRENT|AMT_DRAWINGS_POS_CURRENT|AMT_INST_MIN_REGULARITY|AMT_PAYMENT_CURRENT|AMT_PAYMENT_TOTAL_CURRENT|AMT_RECEIVABLE_PRINCIPAL|AMT_RECIVABLE|AMT_TOTAL_RECEIVABLE|CNT_DRAWINGS_ATM_CURRENT|CNT_DRAWINGS_CURRENT|CNT_DRAWINGS_OTHER_CURRENT|CNT_DRAWINGS_POS_CURRENT|CNT_INSTALMENT_MATURE_CUM|NAME_CONTRACT_STATUS|SK_D

## Sumarizar na visão cliente (Automatizada)

Nesta etapa foi realizada a sumarização automatizada dos dados na visão cliente, com o objetivo de consolidar as informações por contrato (SK_ID_PREV) em diferentes janelas de tempo. Inicialmente, foram selecionadas apenas as colunas relevantes para agregação, excluindo os identificadores e a coluna de controle temporal (SK_ID_CURR, SK_ID_PREV e MONTHS_BALANCE). Em seguida, foram utilizadas as colunas de marcação de período (ultimos_3_meses, ultimos_6_meses, ultimos_12_meses, ultimos_24_meses e ultimos_36_meses) criadas anteriormente, permitindo calcular estatísticas específicas para cada intervalo de tempo. Por meio das funções do módulo pyspark.sql.functions, foram definidas expressões que aplicam operações de soma, média, máximo e mínimo nas variáveis numéricas, condicionadas a cada flag de tempo. Para as variáveis que contêm “DPD”, representando indicadores de atraso, foram calculados apenas os valores máximo e mínimo. Após a definição das expressões de agregação, os dados foram agrupados por SK_ID_PREV e as estatísticas foram aplicadas com o método .agg(), resultando em um novo DataFrame (df_temp_02) que apresenta um resumo consolidado por cliente. Esse processo automatiza a análise temporal, permitindo identificar padrões e variações no comportamento dos clientes ao longo de diferentes períodos.

In [ ]:
from pyspark.sql.functions import col, round, sum, avg, max, min, when

# Definir as colunas para agregação
colunas_agregacao_total = df_temp_01.columns
colunas_agregacao_total.remove('SK_ID_CURR')
colunas_agregacao_total.remove('SK_ID_PREV')
colunas_agregacao_total.remove('MONTHS_BALANCE')

colunas_flags = ['ultimos_3_meses','ultimos_6_meses','ultimos_12_meses','ultimos_24_meses','ultimos_36_meses']

expressoes_agregacao = []

for flag in colunas_flags:
  for coluna in colunas_agregacao_total:
    if 'DPD' in coluna:
      expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MAX_{coluna.upper()}_{flag.upper()}"))
      expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MIN_{coluna.upper()}_{flag.upper()}"))
    else:
      expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_TOT_{coluna.upper()}_{flag.upper()}"))
      expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MED_{coluna.upper()}_{flag.upper()}"))
      expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MAX_{coluna.upper()}_{flag.upper()}"))
      expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MIN_{coluna.upper()}_{flag.upper()}"))

expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicar as expressões de agregação
df_temp_02 = df_temp_01.groupBy("SK_ID_PREV").agg(*expressoes_agregacao).orderBy("SK_ID_PREV")

# Mostrar o DataFrame resultante
df_temp_02.show()

+----------+----------------------------------+----------------------------------+----------------------------------+----------------------------------+----------------------------------------------+----------------------------------------------+----------------------------------------------+----------------------------------------------+-----------------------------------------------+-----------------------------------------------+-----------------------------------------------+-----------------------------------------------+-------------------------------------------+-------------------------------------------+-------------------------------------------+-------------------------------------------+-------------------------------------------------+-------------------------------------------------+-------------------------------------------------+-------------------------------------------------+-----------------------------------------------+--------------------------------------------

## Salvar a tabela sumarizada

Essa parte do código exporta o resultado final do processamento — a tabela com os dados agregados por cliente — para o arquivo credit_card_balance_agg.csv, permitindo que ele seja facilmente compartilhado ou utilizado em outras etapas da análise.

In [ ]:
  df_temp_02 = df_temp_02.repartition(1)
  df_temp_02.write.mode("overwrite").csv("credit_card_balance_agg.csv",header=True)

In [ ]:
!ls -l /content/credit_card_balance_agg.csv


total 221156
-rw-r--r-- 1 root root 226463504 Sep 23 13:22 part-00000-b220581f-b932-4fd2-b01f-b27701a46521-c000.csv
-rw-r--r-- 1 root root         0 Sep 23 13:22 _SUCCESS


In [ ]:
import os

print(os.path.abspath("credit_card_balance_agg.csv"))


/content/credit_card_balance_agg.csv
